# Evaluate ERA-5 downscaling: minimum and maximum temperatures

We used **1981-1991 as training** period and **1991-2010 as reference** period. The results shown in this notebook are based on the model predictions on the reference period.

**Predictors on pressure levels (500, 850)**:
- Geopotential (z)
- Temperature (t)
- Zonal wind (u)
- Meridional wind (v)
- Specific humidity (q)

**Predictors on surface**:
- Surface pressure (p)

**Auxiliary predictors**:
- Elevation from Copernicus EU-DEM v1.1 (dem)
- Day of the year (doy)

Define the predictand and the model to evaluate:

In [1]:
# define the predictand
PREDICTAND = 'tasmean'  # 'tasmin' or 'tasmax'

In [2]:
# model filename to evaluate
FILENAME = 'RandomForest_tasmean_ztuvq_500_850_mslpt2_dem_sa_doy.nc'  # change me!

### Imports

In [3]:
# builtins
import datetime
import warnings
import calendar

# externals
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.stats as stats
from IPython.display import Image
from sklearn.metrics import r2_score

# locals
from downscaleml.main.inputoutput import ERA5_PATH, OBS_PATH, TARGET_PATH, DEM_PATH, MODEL_PATH
from climax.core.utils import plot_loss
from climax.core.dataset import ERA5Dataset
from downscaleml.core.utils import search_files

In [4]:
# mapping from predictands to variable names
NAMES = {'tasmin': 'minimum temperature', 'tasmax': 'maximum temperature', 'pr': 'precipitation', 'tasmean': 'mean temperature'}

### Model architecture

### Load datasets

In [5]:
# model predictions
y_pred = xr.open_dataset(TARGET_PATH.joinpath(PREDICTAND, FILENAME))
y_pred

<xarray.Dataset>
Dimensions:  (time: 1826, y: 5, x: 5)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2020-12-30
  * y        (y) float64 50.52 50.45 50.37 50.3 50.23
  * x        (x) float64 3.715 3.822 3.93 4.038 4.146
Data variables:
    tasmean  (time, y, x) float64 ...

In [6]:
TARGET_PATH.joinpath(PREDICTAND, FILENAME)

PosixPath('/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/larger_alps/ml_Predictions/tasmean/RandomForest_tasmean_ztuvq_500_850_mslpt2_dem_sa_doy.nc')

In [7]:
# target values: observations
y_true = xr.open_dataset(search_files(OBS_PATH.joinpath(PREDICTAND), '.nc$').pop())
y_true

<xarray.Dataset>
Dimensions:            (longitude: 125, latitude: 103, time: 13149)
Coordinates:
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 1985-01-01 ... 2020-12-31
Data variables:
    t2m                (time, latitude, longitude) float32 ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             Norrkoping
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    history:                 Sun Jul 09 20:20:26 2023: cdo remapbil,/mnt/CEPH...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

In [8]:
# subset to time period covered by predictions
y_true = y_true.sel(time=y_pred.time)


In [9]:
y_true = y_true.rename({'t2m': PREDICTAND})
y_true

<xarray.Dataset>
Dimensions:            (longitude: 125, latitude: 103, time: 1826)
Coordinates:
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 2016-01-01 ... 2020-12-30
Data variables:
    tasmean            (time, latitude, longitude) float32 ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             Norrkoping
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    history:                 Sun Jul 09 20:20:26 2023: cdo remapbil,/mnt/CEPH...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

In [10]:
# align datasets and mask missing values in model predictions
y_true, y_pred = xr.align(y_true[PREDICTAND], y_pred[PREDICTAND], join='override')
y_pred = y_pred.where(~np.isnan(y_true), other=np.nan)

## Model validation

In [11]:
y_true

<xarray.DataArray 'tasmean' (time: 1826, latitude: 103, longitude: 125)>
array([[[279.04572, 279.14667, ..., 271.20514, 270.89688],
        [279.28485, 279.49783, ..., 271.2713 , 271.20868],
        ...,
        [287.00836, 287.07147, ..., 280.20163, 279.5809 ],
        [287.3286 , 287.45572, ..., 282.72238, 280.95688]],

       [[281.5531 , 281.49622, ..., 265.88287, 265.60486],
        [281.81924, 281.76315, ..., 266.04095, 266.31705],
        ...,
        [286.61035, 286.7169 , ..., 282.1036 , 281.79193],
        [286.62836, 286.72577, ..., 283.92664, 282.50775]],

       ...,

       [[275.81387, 275.791  , ..., 278.21567, 277.89096],
        [276.07202, 276.11017, ..., 278.00552, 278.72177],
        ...,
        [283.20844, 283.39496, ..., 287.44772, 287.12842],
        [283.4836 , 283.63986, ..., 289.31668, 288.18918]],

       [[276.3766 , 276.4805 , ..., 274.73148, 274.51703],
        [276.55826, 276.66946, ..., 274.81747, 275.04932],
        ...,
        [282.38354, 282.46967, ..., 286.8248 , 286.50635],
        [282.5214 , 282.59323, ..., 288.1987 , 287.06546]]], dtype=float32)
Coordinates:
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 2016-01-01 ... 2020-12-30
Attributes: (12/23)
    standard_name:                            air_temperature
    long_name:                                2 metre temperature
    units:                                    K
    GRIB_dataType:                            an
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_latitudeOfSouthernPoleInDegrees:     -90.0
    GRIB_longitudeOfFirstGridPointInDegrees:  342.514057
    GRIB_longitudeOfSouthernPoleInDegrees:    0.0
    GRIB_name:                                2 metre temperature
    GRIB_shortName:                           2t
    GRIB_units:                               K

In [12]:
y_pred

<xarray.DataArray 'tasmean' (time: 1826, y: 5, x: 5, latitude: 103,
                             longitude: 125)>
array([[[[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
...
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]]]])
Coordinates:
  * time               (time) datetime64[ns] 2016-01-01 ... 2020-12-30
  * y                  (y) float64 50.52 50.45 50.37 50.3 50.23
  * x                  (x) float64 3.715 3.822 3.93 4.038 4.146
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...

In [13]:
y_pred = y_pred - 273.15
y_true = y_true - 273.15

In [14]:
# calculate monthly means
y_pred_mm = y_pred.groupby('time.month').mean(dim=('time'))
y_true_mm = y_true.groupby('time.month').mean(dim=('time'))

In [15]:
y_pred_mm

<xarray.DataArray 'tasmean' (month: 12, y: 5, x: 5, latitude: 103,
                             longitude: 125)>
array([[[[[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
...
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]],

         [[nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          ...,
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan],
          [nan, nan, nan, ..., nan, nan, nan]]]]])
Coordinates:
  * y                  (y) float64 50.52 50.45 50.37 50.3 50.23
  * x                  (x) float64 3.715 3.822 3.93 4.038 4.146
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * month              (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [16]:
y_true_mm

<xarray.DataArray 'tasmean' (month: 12, latitude: 103, longitude: 125)>
array([[[ 4.09377146e+00,  4.04749966e+00,  4.07140541e+00, ...,
         -1.34634688e-01, -7.13638775e-03, -1.46827355e-01],
        [ 4.30060577e+00,  4.34350586e+00,  4.34285164e+00, ...,
         -8.03234458e-01, -6.23013414e-02,  3.59852642e-01],
        [ 4.13744402e+00,  3.90728402e+00,  3.94442487e+00, ...,
         -1.77541065e+00, -1.21269274e+00, -2.00047061e-01],
        ...,
        [ 1.06935873e+01,  1.08360243e+01,  1.09139585e+01, ...,
          1.07534542e+01,  1.08572636e+01,  9.74860764e+00],
        [ 1.09087811e+01,  1.10277672e+01,  1.11121874e+01, ...,
          9.51101208e+00,  9.52820301e+00,  9.01574135e+00],
        [ 1.11201744e+01,  1.12110281e+01,  1.12823668e+01, ...,
          1.11230421e+01,  1.09027882e+01,  9.76825047e+00]],

       [[ 5.07172918e+00,  5.05009365e+00,  5.09870148e+00, ...,
          2.15212321e+00,  2.25022650e+00,  2.15210485e+00],
        [ 5.31383610e+00,  5.40289068e+00,  5.41413116e+00, ...,
          1.34395075e+00,  2.16170359e+00,  2.71566939e+00],
        [ 5.18008280e+00,  4.93566132e+00,  4.98089409e+00, ...,
          3.05534899e-01,  7.73204744e-01,  2.05342007e+00],
...
          1.64594765e+01,  1.66173344e+01,  1.56287088e+01],
        [ 1.44120569e+01,  1.44939384e+01,  1.45587626e+01, ...,
          1.54150801e+01,  1.54836178e+01,  1.51641273e+01],
        [ 1.45742397e+01,  1.46414547e+01,  1.47008915e+01, ...,
          1.67910538e+01,  1.66173115e+01,  1.56364651e+01]],

       [[ 5.12778044e+00,  5.12579775e+00,  5.17670727e+00, ...,
          2.48883200e+00,  2.63479090e+00,  2.52048922e+00],
        [ 5.34922934e+00,  5.42377234e+00,  5.43827152e+00, ...,
          1.73319590e+00,  2.57229853e+00,  3.05518794e+00],
        [ 5.21698713e+00,  5.00982714e+00,  5.05369568e+00, ...,
          7.37529635e-01,  1.31884265e+00,  2.42561102e+00],
        ...,
        [ 1.19777460e+01,  1.21052647e+01,  1.21668701e+01, ...,
          1.30442343e+01,  1.32055502e+01,  1.20463982e+01],
        [ 1.22191277e+01,  1.23172960e+01,  1.23787956e+01, ...,
          1.17039862e+01,  1.17671432e+01,  1.11929998e+01],
        [ 1.24348822e+01,  1.25121231e+01,  1.25669689e+01, ...,
          1.34470119e+01,  1.32318516e+01,  1.19788265e+01]]],
      dtype=float32)
Coordinates:
  * longitude          (longitude) float64 3.715 3.822 3.93 ... 16.96 17.07
  * latitude           (latitude) float64 50.52 50.45 50.37 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * month              (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [17]:
# calculate mean annual cycle
y_pred_ac = y_pred_mm.mean(dim=('x', 'y'))
y_true_ac = y_true_mm.mean(dim=('x', 'y'))

ValueError: 'x' not found in array dimensions ('month', 'latitude', 'longitude')

In [ ]:
# compute daily anomalies
y_pred_anom = ERA5Dataset.anomalies(y_pred, timescale='time.month')
y_true_anom = ERA5Dataset.anomalies(y_true, timescale='time.month')

### Coefficient of determination

In [ ]:
# get predicted and observed daily anomalies
y_pred_av = y_pred_anom.values.flatten()
y_true_av = y_true_anom.values.flatten()

# apply mask of valid pixels
mask = (~np.isnan(y_pred_av) & ~np.isnan(y_true_av))
y_pred_av = y_pred_av[mask]
y_true_av = y_true_av[mask]

# get predicted and observed monthly means
y_pred_mv = y_pred_mm.values.flatten()
y_true_mv = y_true_mm.values.flatten()

# apply mask of valid pixels
mask = (~np.isnan(y_pred_mv) & ~np.isnan(y_true_mv))
y_pred_mv = y_pred_mv[mask]
y_true_mv = y_true_mv[mask]

# calculate coefficient of determination on monthly means
r2_mm = r2_score(y_true_mv, y_pred_mv)
print('R2 on monthly means: {:.2f}'.format(r2_mm))

# calculate coefficient of determination on daily anomalies
r2_anom = r2_score(y_true_av, y_pred_av)
print('R2 on daily anomalies: {:.2f}'.format(r2_anom))

In [ ]:
# scatter plot of observations vs. predictions
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# plot entire dataset
ax.plot(y_true_mv, y_pred_mv, 'o', alpha=.5, markeredgecolor='grey', markerfacecolor='none', markersize=3);

# plot 1:1 mapping line
if PREDICTAND == 'tasmin':
    interval = np.arange(-25, 30, 5)
else:
    interval = np.arange(-15, 45, 5)
ax.plot(interval, interval, color='k', lw=2, ls='--')

# add coefficients of determination
ax.text(interval[-1] - 0.5, interval[0] + 0.5, s='R$^2$ (monthly means)= {:.2f}'.format(r2_mm), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 2.5, s='R$^2$ (daily anomalies) = {:.2f}'.format(r2_anom), ha='right', fontsize=18)


# format axes
ax.set_ylim(interval[0], interval[-1])
ax.set_xlim(interval[0], interval[-1])
ax.set_xticks(interval)
ax.set_xticklabels(interval, fontsize=16)
ax.set_yticks(interval)
ax.set_yticklabels(interval, fontsize=16)
ax.set_xlabel('Observed', fontsize=18)
ax.set_ylabel('Predicted', fontsize=18)
ax.set_title('Monthly mean {} (°C)'.format(NAMES[PREDICTAND]), fontsize=20, pad=10);

# add axis for annual cycle
axins = inset_axes(ax, width="30%", height="40%", loc=2, borderpad=1)
axins.plot(y_pred_ac.values, ls='--', color='k', label='Predicted')
axins.plot(y_true_ac.values, ls='-', color='k', label='Observed')
axins.legend(frameon=False, fontsize=12, loc='lower center');
axins.yaxis.tick_right()
axins.set_yticks(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2))
axins.set_yticklabels(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2), fontsize=12)
axins.set_xticks(np.arange(0, 12))
axins.set_xticklabels([calendar.month_name[i + 1] for i in np.arange(0, 12)], rotation=90, fontsize=12)
axins.set_title('Mean annual cycle', fontsize=14, pad=5);

### Mean error (Bias)

Calculate yearly average bias over entire reference period:

In [ ]:
# yearly average bias over reference period
y_pred_yearly_avg = y_pred.groupby('time.year').mean(dim='time')
y_true_yearly_avg = y_true.groupby('time.year').mean(dim='time')
bias_yearly_avg = y_pred_yearly_avg - y_true_yearly_avg
print('(Model) Yearly average bias of {}: {:.2f}°C'.format(PREDICTAND, bias_yearly_avg.mean().item()))

### Mean absolute error (MAE)

In [ ]:
# mean absolute error over reference period
mae_avg = np.abs(y_pred_yearly_avg - y_true_yearly_avg).mean()
print('(Model) Yearly average MAE of {}: {:.2f}°C'.format(PREDICTAND, mae_avg.mean().item()))

### Root mean squared error (RMSE)

In [ ]:
# root mean squared error over reference period
rmse_avg = np.sqrt(((y_pred_yearly_avg - y_true_yearly_avg) ** 2).mean())
print('(Model) Yearly average RMSE of {}: {:.2f}°C'.format(PREDICTAND, rmse_avg.mean().item()))

In [ ]:
# scatter plot of observations vs. predictions
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# plot entire dataset
ax.plot(y_true_mv, y_pred_mv, 'o', alpha=.5, markeredgecolor='grey', markerfacecolor='none', markersize=3);

# plot 1:1 mapping line
if PREDICTAND == 'tasmin':
    interval = np.arange(-25, 30, 5)
else:
    interval = np.arange(-15, 45, 5)
ax.plot(interval, interval, color='k', lw=2, ls='--')

# add coefficients of determination
ax.text(interval[-1] - 0.5, interval[0] + 0.5, s='R$^2$ (monthly means)= {:.2f}'.format(r2_mm), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 2.5, s='R$^2$ (daily anomalies) = {:.2f}'.format(r2_anom), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 4.5, s='Yearly average bias of {}: {:.2f}°C'.format(PREDICTAND, bias_yearly_avg.mean().item()), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 6.5, s='Yearly average MAE of {}: {:.2f}°C'.format(PREDICTAND, mae_avg.mean().item()), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 8.5, s='Yearly average RMSE of {}: {:.2f}°C'.format(PREDICTAND, rmse_avg.mean().item()), ha='right', fontsize=18)


# format axes
ax.set_ylim(interval[0], interval[-1])
ax.set_xlim(interval[0], interval[-1])
ax.set_xticks(interval)
ax.set_xticklabels(interval, fontsize=16)
ax.set_yticks(interval)
ax.set_yticklabels(interval, fontsize=16)
ax.set_xlabel('CERRA_tasmean', fontsize=18)
ax.set_ylabel('Random_Forest_tasmean', fontsize=18)
ax.set_title('Monthly mean {} (°C)'.format(NAMES[PREDICTAND]), fontsize=20, pad=10);

# add axis for annual cycle
axins = inset_axes(ax, width="35%", height="40%", loc=2, borderpad=1)
axins.plot(y_pred_ac.values, ls='--', color='k', label='RF_tasmean')
axins.plot(y_true_ac.values, ls='-', color='k', label='CERRA')
axins.legend(frameon=False, fontsize=12, loc='lower center');
axins.yaxis.tick_right()
axins.set_yticks(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2))
axins.set_yticklabels(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2), fontsize=12)
axins.set_xticks(np.arange(0, 12))
axins.set_xticklabels([calendar.month_name[i + 1] for i in np.arange(0, 12)], rotation=90, fontsize=12)
axins.set_title('Mean annual cycle', fontsize=14, pad=5);

plt.savefig('mslp.png', dpi=300, bbox_inches='tight')
